In [1]:
import pandas as pd
import numpy as np
from common_variables import *

In [2]:
df_csv = pd.read_csv(full_time_series_path, usecols=['Date','High','Low'], 
    index_col=['Date'], parse_dates=['Date'])
df = df_csv[df_csv.index >= pd.to_datetime(start_date)]
df

,High,Low
Date,,
2010-01-01 00:00:00,1.61673,1.61659
2010-01-01 00:01:00,1.61670,1.61670
2010-01-01 00:02:00,1.61670,1.61665
2010-01-01 00:03:00,1.61670,1.61670
2010-01-01 00:04:00,1.61678,1.61665
...,...,...
2020-09-04 20:55:00,1.32792,1.32761
2020-09-04 20:56:00,1.32822,1.32791
2020-09-04 20:57:00,1.32817,1.32812


In [3]:
df["HLAvg"] = df['High'].add(df['Low']).div(2)
del df['High']
del df['Low']
# Simple Moving Average
df['MA'] = df['HLAvg'].rolling(window=ma_periods).mean()
# Log Returns
df['Returns'] = np.log(df['MA']/df['MA'].shift(1))
df.head(15)

,HLAvg,MA,Returns
Date,,,
2010-01-01 00:00:00,1.616660,NaN,NaN
2010-01-01 00:01:00,1.616700,NaN,NaN
2010-01-01 00:02:00,1.616675,NaN,NaN
2010-01-01 00:03:00,1.616700,NaN,NaN
2010-01-01 00:04:00,1.616715,NaN,NaN
2010-01-01 00:05:00,1.616675,NaN,NaN
2010-01-01 00:06:00,1.616725,NaN,NaN
2010-01-01 00:07:00,1.616725,NaN,NaN
2010-01-01 00:08:00,1.616725,NaN,NaN


In [4]:
df.dropna(how='any', inplace=True)
df = df[df.shape[0] % batch_size:]
df

,HLAvg,MA,Returns
Date,,,
2010-01-01 00:22:00,1.616650,1.616690,-0.000003
2010-01-01 00:23:00,1.616635,1.616683,-0.000005
2010-01-01 00:24:00,1.616640,1.616676,-0.000004
2010-01-01 00:25:00,1.616650,1.616676,0.000000
2010-01-01 00:26:00,1.616660,1.616679,0.000002
...,...,...,...
2020-09-04 20:55:00,1.327765,1.327645,0.000006
2020-09-04 20:56:00,1.328065,1.327686,0.000031
2020-09-04 20:57:00,1.328145,1.327736,0.000038


In [5]:
df_train = df[:- validation_size - test_size]
df_validation = df[- validation_size - test_size - window_size:- test_size]
df_test = df[- test_size - window_size:]
print(f'df_train.shape {df_train.shape}, df_validation.shape {df_validation.shape}, df_test.shape {df_test.shape}')

df_train.shape (3468512, 3), df_validation.shape (262400, 3), df_test.shape (262400, 3)


In [6]:
df_train.to_csv(train_time_series_path)
df_validation.to_csv(validate_time_series_path)
df_test.to_csv(test_time_series_path)